In [1]:
import sys
import os
sys.path.append(os.path.abspath('autoformer'))
from autoformer.model import Autoformer
import pandas as pd
from keras import layers
from keras.models import Model
from sklearn.model_selection import train_test_split

2025-09-17 17:22:06.677607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758117126.694469   35764 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758117126.699840   35764 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758117126.712649   35764 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758117126.712673   35764 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758117126.712675   35764 computation_placer.cc:177] computation placer alr

In [2]:
df= pd.read_csv('data/ETTh1.csv')
df['date']= pd.to_datetime(df['date'])
df.head()

,date,OT
0,2016-07-01 00:00:00,30.531000
1,2016-07-01 01:00:00,27.787001
2,2016-07-01 02:00:00,27.787001
3,2016-07-01 03:00:00,25.044001
4,2016-07-01 04:00:00,21.948000


In [3]:
x= df['date']
y= df['OT']
x_train, x_test, y_train, y_test= train_test_split(x, y, train_size=0.8, shuffle=False)    

In [ ]:
inputs= layers.Input(shape=x_train.shape)
x= Autoformer(d_out= 8,
            d_model= 4,
            n_heads= 16,
            conv_filter= 16)(inputs)

model= Model(inputs= inputs, outputs= x)      
model.compile(optimizer='adam', loss='mse')
model.summary()  